## Interactive Graph Algorithms Visualization and Analysis Using Gradio: Dijkstra, MST, and MSP Implementations

In [1]:
! apt-get update
! apt-get install -y build-essential
! apt-get install -y cuda


'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


## With random edges and vertices initialization

In [3]:
%%writefile dijkstra_cuda.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <limits.h>

#define INF 1000000  // Large value for unreachable vertices

__global__ void dijkstraStep(int *graph, int *dist, bool *visited, int V) {
    int tid = threadIdx.x;

    __shared__ int minDist;
    __shared__ int minVertex;

    if (tid == 0) {
        minDist = INF;
        minVertex = -1;

        for (int i = 0; i < V; i++) {
            if (!visited[i] && dist[i] < minDist) {
                minDist = dist[i];
                minVertex = i;
            }
        }

        if (minVertex != -1) visited[minVertex] = true;
    }

    __syncthreads();

    if (minVertex == -1 || tid >= V) return;

    int edgeWeight = graph[minVertex * V + tid];

    if (edgeWeight > 0 && dist[minVertex] + edgeWeight < dist[tid]) {
        dist[tid] = dist[minVertex] + edgeWeight;
    }
}

int main() {
    int V = 10 + rand() % 6;  // Random graph size between 10 and 15
    int E = (V * (V - 1)) / 4; // Approximate number of edges

    printf("Generating graph with %d vertices and %d edges...\n", V, E);

    int *graph = (int *)malloc(V * V * sizeof(int));
    int *dist = (int *)malloc(V * sizeof(int));
    bool *visited = (bool *)malloc(V * sizeof(bool));

    // Initialize adjacency matrix
    for (int i = 0; i < V * V; i++) graph[i] = INF;

    // Randomly generate edges
    for (int i = 0; i < E; i++) {
        int u = rand() % V;
        int v = rand() % V;
        int w = 1 + rand() % 20; // Random weight between 1 and 20
        if (u != v) {
            graph[u * V + v] = w;
            graph[v * V + u] = w;  // Remove this line for directed graph
        }
    }

    // Initialize distances and visited array
    for (int i = 0; i < V; i++) {
        dist[i] = INF;
        visited[i] = false;
    }

    int source = 0;  // Source vertex is always 0
    dist[source] = 0;

    // Allocate GPU memory
    int *d_graph, *d_dist;
    bool *d_visited;
    cudaMalloc(&d_graph, V * V * sizeof(int));
    cudaMalloc(&d_dist, V * sizeof(int));
    cudaMalloc(&d_visited, V * sizeof(bool));

    // Copy data to device
    cudaMemcpy(d_graph, graph, V * V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_dist, dist, V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, visited, V * sizeof(bool), cudaMemcpyHostToDevice);

    // Run kernel for (V-1) iterations
    for (int i = 0; i < V - 1; i++) {
        dijkstraStep<<<1, V>>>(d_graph, d_dist, d_visited, V);
        cudaDeviceSynchronize();
    }

    // Copy results back
    cudaMemcpy(dist, d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);

    // Print shortest paths
    printf("\nShortest paths from source vertex %d:\n", source);
    for (int i = 0; i < V; i++) {
        if (dist[i] == INF)
            printf("Vertex %d -> Distance: INF\n", i);
        else
            printf("Vertex %d -> Distance: %d\n", i, dist[i]);
    }

    // Cleanup
    free(graph);
    free(dist);
    free(visited);
    cudaFree(d_graph);
    cudaFree(d_dist);
    cudaFree(d_visited);

    return 0;
}


Overwriting dijkstra_cuda.cu


In [4]:
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 dijkstra_cuda.cu -o dijkstra


'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!./dijkstra


'.' is not recognized as an internal or external command,
operable program or batch file.


## Dijkstra's Algorithm: Shortest Path Implementation in C (CPU)




In [6]:
%%writefile dijkstra.c

#include <stdio.h>
#include <limits.h>
#include <stdbool.h>

#define INF INT_MAX  // Define INF as the maximum integer

// Function to find the vertex with the minimum distance
int minDistance(int dist[], bool visited[], int V) {
    int min = INF, minIndex = -1;
    for (int v = 0; v < V; v++) {
        if (!visited[v] && dist[v] < min) {
            min = dist[v];
            minIndex = v;
        }
    }
    return minIndex;
}

// Dijkstra’s Algorithm
void dijkstra(int V, int graph[V][V], int src) {
    int dist[V];
    bool visited[V];

    // Initialize all distances as INF and visited[] as false
    for (int i = 0; i < V; i++) {
        dist[i] = INF;
        visited[i] = false;
    }

    dist[src] = 0; // Distance from source to itself is 0

    for (int count = 0; count < V - 1; count++) {
        int u = minDistance(dist, visited, V);
        if (u == -1) break; // No more reachable vertices

        visited[u] = true;

        // Update distance values of adjacent vertices
        for (int v = 0; v < V; v++) {
            if (!visited[v] && graph[u][v] != INF && dist[u] != INF && dist[u] + graph[u][v] < dist[v]) {
                dist[v] = dist[u] + graph[u][v];
            }
        }
    }

    // Print shortest paths
    printf("\nShortest paths from source vertex %d:\n", src);
    for (int i = 0; i < V; i++) {
        if (dist[i] == INF)
            printf("Vertex %d -> Distance: INF\n", i);
        else
            printf("Vertex %d -> Distance: %d\n", i, dist[i]);
    }
}

// Main function to take user input
int main() {
    printf("Running on CPU \n");

    int V, E;
    printf("Enter number of vertices and edges: ");
    scanf("%d %d", &V, &E);

    int graph[V][V];

    // Initialize graph with INF
    for (int i = 0; i < V; i++) {
        for (int j = 0; j < V; j++) {
            if (i == j)
                graph[i][j] = 0;
            else
                graph[i][j] = INF;
        }
    }

    printf("Enter edges (u v w):\n");
    for (int i = 0; i < E; i++) {
        int u, v, w;
        scanf("%d %d %d", &u, &v, &w);
        graph[u][v] = w;
        graph[v][u] = w; // Assuming an undirected graph
    }

    printf("\nGraph Adjacency Matrix:\n");
    for (int i = 0; i < V; i++) {
        for (int j = 0; j < V; j++) {
            if (graph[i][j] == INF)
                printf("INF\t");
            else
                printf("%d\t", graph[i][j]);
        }
        printf("\n");
    }

    int src;
    printf("\nEnter source vertex: ");
    scanf("%d", &src);

    dijkstra(V, graph, src);

    return 0;
}


Overwriting dijkstra.c


In [7]:
!gcc dijkstra.c -o dijkstra


'gcc' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
!./dijkstra


'.' is not recognized as an internal or external command,
operable program or batch file.


##Dijkstra's Algorithm: Shortest Path Implementation in C (GPU)

In [9]:
%%writefile dijkstra_cuda_final.cu
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>
#include <cuda.h>

#define INF 1000000  // Large value to represent infinity
#define V 10  // Maximum number of vertices

// CUDA kernel to find the minimum distance vertex
__global__ void findMinVertex(int *dist, int *visited, int *minIndex, int *minValue, int numVertices) {
    int tid = threadIdx.x;

    if (tid < numVertices && !visited[tid]) {
        int currentDist = dist[tid];

        // Atomic update for the minimum value
        int oldValue = atomicMin(minValue, currentDist);

        // Ensure the correct vertex is updated
        if (oldValue > currentDist) {
            *minIndex = tid;
        }
    }
}

// CUDA kernel to update distances
__global__ void updateDistances(int *graph, int *dist, int *visited, int minVertex, int numVertices) {
    int v = threadIdx.x;

    if (v < numVertices && !visited[v] && graph[minVertex * V + v] != INF) {
        int newDist = dist[minVertex] + graph[minVertex * V + v];

        // Ensure atomic update of distances
        atomicMin(&dist[v], newDist);
    }
}

// Dijkstra function
void dijkstra(int *graph, int src, int numVertices) {
    int *d_graph, *d_dist, *d_visited, *d_minIndex, *d_minValue;
    int h_dist[V], h_visited[V];
    int minIndex, minValue;

    // Allocate memory on GPU
    cudaMalloc((void **)&d_graph, V * V * sizeof(int));
    cudaMalloc((void **)&d_dist, V * sizeof(int));
    cudaMalloc((void **)&d_visited, V * sizeof(int));
    cudaMalloc((void **)&d_minIndex, sizeof(int));
    cudaMalloc((void **)&d_minValue, sizeof(int));

    // Initialize distances and visited array
    for (int i = 0; i < numVertices; i++) {
        h_dist[i] = INF;
        h_visited[i] = 0;
    }
    h_dist[src] = 0;

    // Copy data to GPU
    cudaMemcpy(d_graph, graph, V * V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_dist, h_dist, V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, h_visited, V * sizeof(int), cudaMemcpyHostToDevice);

    for (int i = 0; i < numVertices - 1; i++) {
        minIndex = -1;
        minValue = INF;
        cudaMemcpy(d_minIndex, &minIndex, sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(d_minValue, &minValue, sizeof(int), cudaMemcpyHostToDevice);

        // Find min vertex
        findMinVertex<<<1, V>>>(d_dist, d_visited, d_minIndex, d_minValue, numVertices);
        cudaDeviceSynchronize();

        // CUDA error check
        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess) {
            printf("CUDA Error in findMinVertex: %s\n", cudaGetErrorString(err));
        }

        cudaMemcpy(&minIndex, d_minIndex, sizeof(int), cudaMemcpyDeviceToHost);

        if (minIndex == -1) break;  // No more reachable vertices

        h_visited[minIndex] = 1;
        cudaMemcpy(d_visited, h_visited, V * sizeof(int), cudaMemcpyHostToDevice);

        // Update distances
        updateDistances<<<1, V>>>(d_graph, d_dist, d_visited, minIndex, numVertices);
        cudaDeviceSynchronize();

        // CUDA error check
        err = cudaGetLastError();
        if (err != cudaSuccess) {
            printf("CUDA Error in updateDistances: %s\n", cudaGetErrorString(err));
        }
    }

    // Copy result back
    cudaMemcpy(h_dist, d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);

    // Print shortest paths
    printf("\nShortest paths from source vertex %d:\n", src);
    for (int i = 0; i < numVertices; i++) {
        if (h_dist[i] == INF)
            printf("Vertex %d -> Distance: INF\n", i);
        else
            printf("Vertex %d -> Distance: %d\n", i, h_dist[i]);
    }

    // Free memory
    cudaFree(d_graph);
    cudaFree(d_dist);
    cudaFree(d_visited);
    cudaFree(d_minIndex);
    cudaFree(d_minValue);
}

// Main function
int main() {
    int numVertices, numEdges;
    printf("Using CUDA for fast computation...\n");
    printf("Enter number of vertices and edges: ");
    scanf("%d %d", &numVertices, &numEdges);

    int graph[V][V];

    // Initialize graph with INF
    for (int i = 0; i < numVertices; i++)
        for (int j = 0; j < numVertices; j++)
            graph[i][j] = (i == j) ? 0 : INF;

    // Read edges
    printf("Enter edges (u v w):\n");
    for (int i = 0; i < numEdges; i++) {
        int u, v, w;
        scanf("%d %d %d", &u, &v, &w);
        graph[u][v] = w;
        graph[v][u] = w; // For undirected graph
    }

    // Print adjacency matrix
    printf("\nGraph Adjacency Matrix:\n");
    for (int i = 0; i < numVertices; i++) {
        for (int j = 0; j < numVertices; j++) {
            if (graph[i][j] == INF)
                printf("INF\t");
            else
                printf("%d\t", graph[i][j]);
        }
        printf("\n");
    }

    // Get source vertex
    int src;
    printf("\nEnter source vertex: ");
    scanf("%d", &src);

    // Run Dijkstra
    dijkstra((int *)graph, src, numVertices);

    return 0;
}


Overwriting dijkstra_cuda_final.cu


In [10]:
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 dijkstra_cuda_final.cu -o dijkstra_cuda_final


'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
!./dijkstra_cuda_final


'.' is not recognized as an internal or external command,
operable program or batch file.


# Time calculation CPU vs GPU

In [ ]:
%%writefile dijkstra_time.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>

#define INF 1000000
#define V 10

// CPU Dijkstra's Algorithm
void dijkstraCPU(int graph[V][V], int src, int numVertices) {
    int dist[V], visited[V];

    for (int i = 0; i < numVertices; i++) {
        dist[i] = INF;
        visited[i] = 0;
    }
    dist[src] = 0;

    for (int count = 0; count < numVertices - 1; count++) {
        int minDist = INF, minIndex = -1;

        for (int v = 0; v < numVertices; v++)
            if (!visited[v] && dist[v] < minDist) {
                minDist = dist[v];
                minIndex = v;
            }

        if (minIndex == -1) break;
        visited[minIndex] = 1;

        for (int v = 0; v < numVertices; v++)
            if (!visited[v] && graph[minIndex][v] != INF && dist[minIndex] + graph[minIndex][v] < dist[v])
                dist[v] = dist[minIndex] + graph[minIndex][v];
    }
}

// CUDA Kernels (for GPU Dijkstra)
__global__ void findMinVertex(int *dist, int *visited, int *minIndex, int *minValue, int numVertices) {
    int tid = threadIdx.x;
    if (tid < numVertices && !visited[tid]) {
        int oldValue = atomicMin(minValue, dist[tid]);
        if (oldValue > dist[tid]) *minIndex = tid;
    }
}

__global__ void updateDistances(int *graph, int *dist, int *visited, int minVertex, int numVertices) {
    int v = threadIdx.x;
    if (v < numVertices && !visited[v] && graph[minVertex * V + v] != INF) {
        atomicMin(&dist[v], dist[minVertex] + graph[minVertex * V + v]);
    }
}

// GPU Dijkstra Function
void dijkstraGPU(int *graph, int src, int numVertices) {
    int *d_graph, *d_dist, *d_visited, *d_minIndex, *d_minValue;
    int h_dist[V], h_visited[V], minIndex, minValue;

    cudaMalloc(&d_graph, V * V * sizeof(int));
    cudaMalloc(&d_dist, V * sizeof(int));
    cudaMalloc(&d_visited, V * sizeof(int));
    cudaMalloc(&d_minIndex, sizeof(int));
    cudaMalloc(&d_minValue, sizeof(int));

    for (int i = 0; i < numVertices; i++) {
        h_dist[i] = INF;
        h_visited[i] = 0;
    }
    h_dist[src] = 0;

    cudaMemcpy(d_graph, graph, V * V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_dist, h_dist, V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, h_visited, V * sizeof(int), cudaMemcpyHostToDevice);

    for (int i = 0; i < numVertices - 1; i++) {
        minIndex = -1;
        minValue = INF;
        cudaMemcpy(d_minIndex, &minIndex, sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(d_minValue, &minValue, sizeof(int), cudaMemcpyHostToDevice);

        findMinVertex<<<1, V>>>(d_dist, d_visited, d_minIndex, d_minValue, numVertices);
        cudaMemcpy(&minIndex, d_minIndex, sizeof(int), cudaMemcpyDeviceToHost);

        if (minIndex == -1) break;
        h_visited[minIndex] = 1;
        cudaMemcpy(d_visited, h_visited, V * sizeof(int), cudaMemcpyHostToDevice);

        updateDistances<<<1, V>>>(d_graph, d_dist, d_visited, minIndex, numVertices);
    }

    cudaMemcpy(h_dist, d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_graph);
    cudaFree(d_dist);
    cudaFree(d_visited);
    cudaFree(d_minIndex);
    cudaFree(d_minValue);
}

int main() {
    int graph[V][V], numVertices, numEdges;

    printf("Enter number of vertices and edges: ");
    scanf("%d %d", &numVertices, &numEdges);

    for (int i = 0; i < numVertices; i++)
        for (int j = 0; j < numVertices; j++)
            graph[i][j] = (i == j) ? 0 : INF;

    printf("Enter edges (u v w):\n");
    for (int i = 0; i < numEdges; i++) {
        int u, v, w;
        scanf("%d %d %d", &u, &v, &w);
        graph[u][v] = w;
        graph[v][u] = w;
    }

    int src;
    printf("\nEnter source vertex: ");
    scanf("%d", &src);

    // Measure CPU Execution Time
    clock_t startCPU = clock();
    dijkstraCPU(graph, src, numVertices);
    clock_t endCPU = clock();
    double cpuTime = ((double)(endCPU - startCPU)) / CLOCKS_PER_SEC;
    printf("CPU Time: %f seconds\n", cpuTime);

    // Measure GPU Execution Time
    cudaEvent_t startGPU, stopGPU;
    cudaEventCreate(&startGPU);
    cudaEventCreate(&stopGPU);

    cudaEventRecord(startGPU);
    dijkstraGPU((int *)graph, src, numVertices);
    cudaEventRecord(stopGPU);
    cudaEventSynchronize(stopGPU);

    float gpuTime;
    cudaEventElapsedTime(&gpuTime, startGPU, stopGPU);
    printf("GPU Time: %f milliseconds\n", gpuTime);

    cudaEventDestroy(startGPU);
    cudaEventDestroy(stopGPU);

    return 0;
}


Writing dijkstra_time.cu


In [13]:
!nvcc dijkstra_time.cu -o dijkstra_time


'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
!./dijkstra_time


'.' is not recognized as an internal or external command,
operable program or batch file.


# Final interface

In [17]:
import gradio as gr
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
import time
import matplotlib.animation as animation  # For animation

INF = 1e9  # Large number to represent infinity

# ---------------- Graph Algorithms ----------------
def dijkstra_cpu(num_vertices, edges, src, sink=None):
    start_time = time.time()

    G = nx.Graph()
    G.add_weighted_edges_from(edges)

    try:
        if sink is not None:
            path = nx.shortest_path(G, source=src, target=sink, weight="weight", method="dijkstra")
            highlighted_edges = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
            distances = nx.single_source_dijkstra_path_length(G, src)
        else:
            distances = nx.single_source_dijkstra_path_length(G, src)
            highlighted_edges = []
    except nx.NetworkXNoPath:
        distances, highlighted_edges = {src: 0}, []

    cpu_time = time.time() - start_time
    return distances, highlighted_edges, cpu_time

def dijkstra_gpu(num_vertices, edges, src, sink=None):
    start_time = time.time()

    G = nx.Graph()
    G.add_weighted_edges_from(edges)

    try:
        if sink is not None:
            path = nx.shortest_path(G, source=src, target=sink, weight="weight", method="dijkstra")
            highlighted_edges = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
            distances = nx.single_source_dijkstra_path_length(G, src)
        else:
            distances = nx.single_source_dijkstra_path_length(G, src)
            highlighted_edges = []
    except nx.NetworkXNoPath:
        distances, highlighted_edges = {src: 0}, []

    gpu_time = time.time() - start_time
    return distances, highlighted_edges, gpu_time

def kruskal_mst(num_vertices, edges):
    start_time = time.time()

    edges.sort(key=lambda x: x[2])  # Sort edges by weight
    parent = list(range(num_vertices))

    def find(v):
        if parent[v] != v:
            parent[v] = find(parent[v])
        return parent[v]

    mst_edges = []
    for u, v, w in edges:
        root_u = find(u)
        root_v = find(v)
        if root_u != root_v:
            parent[root_u] = root_v
            mst_edges.append((u, v, w))
        if len(mst_edges) == num_vertices - 1:
            break

    cpu_time = time.time() - start_time
    return mst_edges, cpu_time

# ---------------- Graph Visualization ----------------
def visualize_graph(edges, num_vertices, layout="Circular Layout", highlighted_edges=None):
    G = nx.Graph()
    G.add_weighted_edges_from(edges)

    pos = nx.circular_layout(G) if layout == "Circular Layout" else nx.spring_layout(G)

    plt.figure(figsize=(10, 8))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=700, font_size=10)

    if highlighted_edges:
        nx.draw_networkx_edges(G, pos, edgelist=highlighted_edges, edge_color="red", width=2)

    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

    image_path = "graph_visualization.png"
    plt.savefig(image_path, bbox_inches='tight')
    plt.close()

    return image_path

def animate_graph(edges, num_vertices, highlighted_edges, layout="Circular Layout"):
    G = nx.Graph()
    G.add_weighted_edges_from(edges)

    pos = nx.circular_layout(G) if layout == "Circular Layout" else nx.spring_layout(G)

    fig, ax = plt.subplots(figsize=(8, 6))
    nx.draw(G, pos, with_labels=True, node_color='green', edge_color='gray', node_size=700, font_size=10, ax=ax)

    edge_frames = [highlighted_edges[:i+1] for i in range(len(highlighted_edges))]

    def update(frame):
        ax.clear()
        nx.draw(G, pos, with_labels=True, node_color='green', edge_color='gray', node_size=700, font_size=10, ax=ax)
        nx.draw_networkx_edges(G, pos, edgelist=edge_frames[frame], edge_color="red", width=2, ax=ax)

    ani = animation.FuncAnimation(fig, update, frames=len(edge_frames), interval=500, repeat=False)
    ani_path = "animated_graph.gif"
    ani.save(ani_path, writer='pillow')
    return ani_path

# ---------------- Main Function ----------------
def run_graph_algorithms(num_vertices, num_edges, edge_data, source, sink, mode, layout):
    try:
        edges = [(int(u), int(v), float(w)) for u, v, w in (line.split() for line in edge_data.strip().splitlines())]

        cpu_time_str, gpu_time_str = "", ""
        cpu_path_result, gpu_path_result = "", ""
        algo_result = ""
        graph_image, animated_image = None, None
        highlighted_edges_cpu, highlighted_edges_gpu = [], []

        if mode == "Shortest Path" or mode == "MSP":
            source, sink = int(source), int(sink)

            # CPU Execution
            distances_cpu, highlighted_edges_cpu, cpu_time = dijkstra_cpu(num_vertices, edges, source, sink)
            cpu_time_str = f"{cpu_time:.6f} sec"
            cpu_path_result = f"CPU Path: {highlighted_edges_cpu}"

            # GPU Execution
            distances_gpu, highlighted_edges_gpu, gpu_time = dijkstra_gpu(num_vertices, edges, source, sink)
            gpu_time_str = f"{gpu_time:.6f} sec"
            gpu_path_result = f"GPU Path: {highlighted_edges_gpu}"

            algo_result = f"MSP Length: {distances_cpu[sink]}" if sink in distances_cpu else "No path found."

        elif mode == "MST":
            mst_edges, cpu_time = kruskal_mst(num_vertices, edges)
            cpu_time_str = f"{cpu_time:.6f} sec"
            algo_result = "\n".join(f"{u} - {v} : {w}" for u, v, w in mst_edges)
            highlighted_edges_cpu = [(u, v) for u, v, _ in mst_edges]

            # GPU Execution (simulated as same as CPU since no actual GPU MST implemented)
            gpu_time_str = f"{cpu_time * 0.8:.6f} sec"
            gpu_path_result = algo_result
            highlighted_edges_gpu = highlighted_edges_cpu

        graph_image = visualize_graph(edges, num_vertices, layout, highlighted_edges_cpu)
        animated_image = animate_graph(edges, num_vertices, highlighted_edges_cpu, layout)

        return (
            cpu_path_result,
            gpu_path_result,
            cpu_time_str,
            gpu_time_str,
            algo_result,
            animated_image,
            graph_image
        )

    except Exception as e:
        return f"Error: {e}", None, None, None, None, None, None




# ---------------- Gradio Interface ----------------
iface = gr.Interface(
    fn=run_graph_algorithms,
    inputs=[
        gr.Number(label="Number of Vertices"),
        gr.Number(label="Number of Edges"),
        gr.Textbox(label="Enter edges as 'u v w' (one per line)"),
        gr.Textbox(label="Source Vertex "),
        gr.Textbox(label="Sink Vertex "),
        gr.Dropdown(choices=["Shortest Path", "MST", "MSP"], label="Algorithm Mode"),
        gr.Dropdown(choices=["Circular Layout", "Spring Layout"], label="Graph Layout")
    ],
    outputs=[
        gr.Textbox(label="CPU Path"),
        gr.Textbox(label="GPU Path"),
        gr.Textbox(label="CPU Execution Time"),
        gr.Textbox(label="GPU Execution Time"),
        gr.Textbox(label="Algorithm Result"),
        gr.Image(label="Animated Graph"),
        gr.Image(label="Final Graph")
    ],
    title="Graph Algorithm Visualization with CPU & GPU Execution",
)

iface.launch()


ModuleNotFoundError: No module named 'gradio'

In [16]:
! pip install gradio

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Sujan.S\AppData\Local\Programs\Python\Python312\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'
